pip install openpyxl


In [1]:
import pandas as pd
import numpy as np

In [2]:
excel_file = '../data/clean_data/lapd_ripa_raw_data_02012021_-_08312021.xlsx'

In [3]:
raw_data = pd.read_excel(excel_file, sheet_name=None)

In [4]:
raw_data.keys()

dict_keys(['MAIN', 'Officer', 'ACTIONS TAKEN', 'BASIS FOR SEARCH', 'Reason For Stop', 'Result of Stop', 'Person', 'Perceived Race', 'Perceived Disability', 'Person Contraband Discovered ', 'Result of Stop and Race'])

In [5]:
raw_data['MAIN']['Stop Duration'].max()

380

In [6]:
for sheet in raw_data.keys():
    if "Stop ID" in raw_data[sheet].columns:
        print("{} \t\t{} rows  \t {} unique stops ".format(sheet, len(raw_data[sheet]), len(raw_data[sheet]["Stop ID"].unique())))

MAIN 		1425 rows  	 1425 unique stops 
ACTIONS TAKEN 		1663 rows  	 1425 unique stops 
Reason For Stop 		1447 rows  	 1425 unique stops 
Result of Stop 		1519 rows  	 1425 unique stops 
Person 		1440 rows  	 1425 unique stops 
Perceived Race 		1462 rows  	 1425 unique stops 
Perceived Disability 		1443 rows  	 1425 unique stops 
Person Contraband Discovered  		1449 rows  	 1425 unique stops 
Result of Stop and Race 		1573 rows  	 1425 unique stops 


### Identify and drop stops where multiple people are involved in stops

Because we can't identify the actions taken per person when multiple people are stopped, or who was intended to be stopped in the case of a driver with passengers, we can't reliably use this data to say whether racial profiling occurred.

#### Identifying stops with multiple people

In [7]:
person, race, disability, stops = raw_data['Person'], raw_data['Perceived Race'], raw_data['Perceived Disability'], raw_data['MAIN']

In [8]:
# Mult rows in Persons df
mult_people_stopped = person['Stop ID'].duplicated(keep=False)
bad_stops1 = person[mult_people_stopped]['Stop ID'].unique()

# Mult rows in percieved Race df
mult_people_stopped = race['Stop ID'].duplicated(keep=False)
bad_stops2 = race[mult_people_stopped]['Stop ID'].unique()

# Mult rows in percieved Disability df
mult_people_stopped = disability['Stop ID'].duplicated(keep=False)
bad_stops3 = disability[mult_people_stopped]['Stop ID'].unique()

# Combine stop ids
all_bad_stops = np.union1d(np.union1d(bad_stops1, bad_stops2), bad_stops3)

In [9]:
len(bad_stops1)

14

In [10]:
len(bad_stops2)

33

In [11]:
len(bad_stops3)

17

In [12]:
len(all_bad_stops)

36

In [13]:
len(all_bad_stops) / len(stops)

0.02526315789473684

So we lose 2.5% of our data by dropping stops with multiple people.

Note that a single row in the person sheet can have a one-to-many mapping with percieved race or percieved disability. Presumably, this is an error in the data collection. The officer may have remembered to jot down the percieved race of everyone but not the percieved age & gender of everyone. Take, for example, stop 1752 where one person's gender & age was jotted down, but then multiple races are jotted down for the same stop.

In [14]:
person.query('`Stop ID` == 1752')

,Stop ID,Perceived Limited English,Perceived Age,Perceived Gender,Gender Nonconforming,Perceived LGBT,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
857,1752,No,25,Male,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
race.query('`Stop ID` == 1752')

,Stop ID,Perceived Race
872,1752,White
873,1752,Asian
874,1752,Black/African American


In [16]:
disability.query('`Stop ID` == 1752')

,Stop ID,Perceived Or Known Disability
858,1752,None


#### Dropping stops with multiple people

In [17]:
# Drop unnamed columns. TODO: Find more systematic way of doing this. 
person = person.iloc[:, :6]

In [18]:
# Drop stops with multiple people stopped
person = person[~person['Stop ID'].isin(all_bad_stops)]

In [19]:
person.shape

(1389, 6)

In [21]:
df = person.merge(race, how='left', on='Stop ID')

In [22]:
df = df.merge(disability, how='left', on='Stop ID')

In [23]:
df.shape

(1389, 8)

In [24]:
df = df.merge(raw_data['MAIN'], how='left', on='Stop ID')

In [25]:
df.shape

(1389, 12)

In [26]:
df.columns

Index(['Stop ID', 'Perceived Limited English', 'Perceived Age',
       'Perceived Gender', 'Gender Nonconforming', 'Perceived LGBT',
       'Perceived Race', 'Perceived Or Known Disability', 'Stop Date Time',
       'Stop Duration', 'Stop In Response To Calls for Service', 'Street'],
      dtype='object')

#### Actions taken dataframe

In [27]:
actions = raw_data['ACTIONS TAKEN']

In [28]:
actions.columns

Index(['Stop ID', 'Action Taken During Stop', 'Person Search Consent Given',
       'Property Search Consent Given'],
      dtype='object')

In [29]:
actions = actions[~actions['Stop ID'].isin(all_bad_stops)]

In [30]:
# Get all actions for a stop
actions = pd.get_dummies(actions).groupby('Stop ID').sum()

In [31]:
# Remove columns for saying 'no' to request for consent to search. It's redundant.
to_drop = actions.columns[actions.columns.str.endswith('Given_No')]

In [32]:
actions = actions.drop(to_drop, axis=1)

In [33]:
actions.shape

(1389, 17)

### Reason for stop

In [35]:
motive = raw_data['Reason For Stop']

In [36]:
motive = pd.get_dummies(motive)

### Result of stop

In [37]:
result = raw_data['Result of Stop']

In [38]:
result = pd.get_dummies(result)

### Contraband

In [39]:
contraband = raw_data['Person Contraband Discovered ']

In [40]:
contraband = pd.get_dummies(contraband)

### Merge

In [41]:
df = df.merge(actions, how='left', on='Stop ID')

In [42]:
df = df.merge(motive, how='left', on='Stop ID')

In [43]:
df = df.merge(result, how='left', on='Stop ID')

In [44]:
df = df.merge(contraband, how='left', on='Stop ID')

#### It's all merged! Now to simplify the above to make it digestible...

In [ ]:
for sid in df['Stop ID'].unique()[:30]:
    sid_acts = actions[actions['Stop ID'] == sid]
    if sid_acts.shape[0] > 1:
        print(sid_acts)

In [ ]:
for sheet in raw_data:
    print('sheet', sheet)
    for c in raw_data[sheet].columns:
        print('\t' + c)

In [ ]:
raw_data['Reason For Stop']

In [ ]:
data = raw_data['MAIN']
sheets_to_merge=['ACTIONS TAKEN', 'BASIS FOR SEARCH', 'Reason For Stop', 
                 'Result of Stop', 'Person', 'Perceived Race', 
                 'Perceived Disability', 'Person Contraband Discovered ']
for sheet in sheets_to_merge:
    data = data.merge(raw_data[sheet], how="inner", on="Stop ID")

In [ ]:
data.keys()

In [ ]:
data

In [ ]:
len(data)

In [ ]:
len(data["Stop ID"].unique())

In [ ]:
data[data["Stop ID"] == 85].drop_duplicates()

In [ ]:
data["Result Of Stop"].value_counts()

In [ ]:
data["Stop ID"].value_counts()

In [ ]:
data.drop_duplicates()["Stop ID"].value_counts()

In [ ]:
data[data["Stop ID"] == 2217]